In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import os
import sys
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Get the absolute path to the 'src' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..")) #change when running in a different directory
src_path = os.path.join(project_root, "src")

# Add 'src' to system path
if src_path not in sys.path:
    sys.path.append(src_path)

from create_Dataframe import createDataframe as create_df

# Base Case Summer
### Loading Data

In [3]:
merged_data_summer = create_df('summer')

c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Case Global Variables

In [6]:
price = merged_data_summer["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_summer['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_base_case_summer")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 0 #no heatpump in this case
power_hp = [0] * len(price)  # Heat pump power is zero in this case

### Appliances

In [8]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_summer,model)

duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_summer,model)

binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_summer,model, duration_wm, wm_start)

charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_summer,model)

penalty_cost, level_bin, levels, penalty_per_level, total_demand = peak_prices(Time_interval,merged_data_summer,model,
            inflexible_demand, binary_wm, binary_dishwasher, binary_dryer, charging_ev, power_wm, 
            power_dishwasher, max_power_ev, power_dryer, max_power_hp, power_hp)

### Run Optimization

In [7]:
# Electricity cost
electricity_cost = gp.quicksum(
    (price[t] *
        (merged_data_summer['Inflexible_Demand_(kWh)'][t] +
        power_dishwasher * binary_dishwasher[t] +
        power_wm * binary_wm[t] +
        power_dryer * binary_dryer[t] +
        charging_ev[t])) for t in range(Time_interval)
)
# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 33648 rows, 16124 columns and 96922 nonzeros
Model fingerprint: 0xdae1bd4a
Variable types: 3360 continuous, 12764 integer (12764 binary)
Coefficient statistics:
  Matrix range     [3e-01, 3e+01]
  Objective range  [2e-03, 7e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [7e-02, 7e+01]
Presolve removed 26364 rows and 9759 columns (presolve time = 7s)...
Presolve removed 26364 rows and 9759 columns
Presolve time: 7.28s
Presolved: 7284 rows, 6365 columns, 53568 nonzeros
Variable types: 680 continuous, 5685 integer (5685 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8235554e+01   6.654612e+02   0.000000e+00      7s
    2836    3.8435458e+01   0.000000e+00

### Save Data

In [ ]:
# Prepare results DataFrame
results_df = merged_data_summer.copy()
results_df['Total_Demand'] = [total_demand[t].X for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]

# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [
    int(np.argmax([level_bin[t][i].X for i in range(len(levels)-1)]))
    for t in range(len(results_df))
]

# Add penalty cost per hour based on demand level
results_df['Penalty_Cost'] = [
    penalty_per_level[dl] for dl in results_df['Demand_Level']
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = results_df['Total_Demand'] * results_df['Spotmarket_(EUR/kWh)']

# Add total cost per hour (electricity + penalty)
results_df['Total_Cost'] = results_df['Electricity_Cost'] + results_df['Penalty_Cost']

# Save to CSV
results_csv_path = os.path.join(project_root + "/results/", "results_basecase_summer.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project/results/results_basecase_summer.csv


# Base Case Winter
### Loading Data

In [9]:
merged_data_winter = create_df('winter')

c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Case Global Variables

In [10]:
price = merged_data_winter["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_winter['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_base_case_winter")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 0 #no heatpump in this case
power_hp = [0] * len(price)  # Heat pump power is zero in this case

### Appliances

In [11]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_winter,model)

duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_winter,model)

binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_winter,model, duration_wm, wm_start)

charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_winter,model)

penalty_cost, level_bin, levels, penalty_per_level, total_demand = peak_prices(Time_interval,merged_data_winter,model,
            inflexible_demand, binary_wm, binary_dishwasher, binary_dryer, charging_ev, power_wm, 
            power_dishwasher, max_power_ev, power_dryer, max_power_hp, power_hp)

### Objective Function and Optimization

In [13]:
# Electricity cost
electricity_cost = gp.quicksum(
    (price[t] *
        (merged_data_winter['Inflexible_Demand_(kWh)'][t] +
        power_dishwasher * binary_dishwasher[t] +
        power_wm * binary_wm[t] +
        power_dryer * binary_dryer[t] +
        charging_ev[t])) for t in range(Time_interval)
)
# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 33648 rows, 16124 columns and 96922 nonzeros
Model fingerprint: 0xc24ad96d
Variable types: 3360 continuous, 12764 integer (12764 binary)
Coefficient statistics:
  Matrix range     [3e-01, 3e+01]
  Objective range  [2e-04, 7e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [9e-02, 7e+01]
Presolve removed 26371 rows and 9765 columns (presolve time = 6s)...
Presolve removed 26371 rows and 9765 columns
Presolve time: 6.40s
Presolved: 7277 rows, 6359 columns, 53502 nonzeros
Variable types: 678 continuous, 5681 integer (5681 binary)
Found heuristic solution: objective 72.9329247

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8404385e+01   6.570164e+02   0.000000e+00  

### Save Data

In [14]:
# Prepare results DataFrame
results_df = merged_data_winter.copy()
results_df['Total_Demand'] = [total_demand[t].X for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]

# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [
    int(np.argmax([level_bin[t][i].X for i in range(len(levels)-1)]))
    for t in range(len(results_df))
]

# Add penalty cost per hour based on demand level
results_df['Penalty_Cost'] = [
    penalty_per_level[dl] for dl in results_df['Demand_Level']
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = results_df['Total_Demand'] * results_df['Spotmarket_(EUR/kWh)']

# Add total cost per hour (electricity + penalty)
results_df['Total_Cost'] = results_df['Electricity_Cost'] + results_df['Penalty_Cost']

# Save to CSV
results_csv_path = os.path.join(project_root + "/results/", "results_basecase_winter.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project/results/results_basecase_winter.csv


# Case 1 Winter - with heatpump 
### Loading Data

In [12]:
merged_data_winter_case_1 = create_df('winter')

c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Case Global Variables

In [13]:
price = merged_data_winter_case_1["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_winter_case_1['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_1_winter")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 8

### Appliances

In [ ]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_winter_case_1,model)

duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_winter_case_1,model)

binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_winter_case_1,model, duration_wm, wm_start)

charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_winter_case_1,model)

power_hp = heat_pump(Time_interval,merged_data_winter_case_1,model,max_power_hp)

penalty_cost, level_bin, levels, penalty_per_level, total_demand = peak_prices(Time_interval,merged_data_winter_case_1,model,
            inflexible_demand, binary_wm, binary_dishwasher, binary_dryer, charging_ev, power_wm, 
            power_dishwasher, max_power_ev, power_dryer, max_power_hp,power_hp)


In [16]:
# Electricity cost
electricity_cost = gp.quicksum(
    (price[t] *
        (merged_data_winter_case_1['Inflexible_Demand_(kWh)'][t] +
        power_dishwasher * binary_dishwasher[t] +
        power_wm * binary_wm[t] +
        power_dryer * binary_dryer[t] +
        charging_ev[t] +
        power_hp[t])) for t in range(Time_interval)
)

# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 44400 rows, 24188 columns and 125142 nonzeros
Model fingerprint: 0x2f704db2
Variable types: 8064 continuous, 16124 integer (16124 binary)
Coefficient statistics:
  Matrix range     [3e-01, 4e+01]
  Objective range  [2e-04, 7e-01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [5e-05, 1e+02]
Presolve removed 31244 rows and 9990 columns (presolve time = 10s)...
Presolve removed 31244 rows and 9990 columns
Presolve time: 10.00s
Presolved: 13156 rows, 14198 columns, 61153 nonzeros
Variable types: 4272 continuous, 9926 integer (9926 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8404385e+01   1.972808e+03   0.000000e+00     10s
    5197    5.0060612e+01   0.0000

### Save Data

In [18]:
# Prepare results DataFrame
results_df = merged_data_winter_case_1.copy()
results_df['Total_Demand'] = [total_demand[t].X for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
results_df['Heatpump_Power'] = [power_hp[t].X for t in range(len(results_df))]

# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [
    int(np.argmax([level_bin[t][i].X for i in range(len(levels)-1)]))
    for t in range(len(results_df))
]

# Add penalty cost per hour based on demand level
results_df['Penalty_Cost'] = [
    penalty_per_level[dl] for dl in results_df['Demand_Level']
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = results_df['Total_Demand'] * results_df['Spotmarket_(EUR/kWh)']

# Add total cost per hour (electricity + penalty)
results_df['Total_Cost'] = results_df['Electricity_Cost'] + results_df['Penalty_Cost']

# Save to CSV
results_csv_path = os.path.join(project_root + "/results/", "results_case_1_winter.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project/results/results_case_1_winter.csv


# Case 2 Summer 
## PV Generation without feed-in
### Loading Data

In [19]:
merged_data_summer_case_2 = create_df('summer')

c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Case Global Variables

In [20]:
price = merged_data_summer_case_2["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_summer_case_2['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_2_summer")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 0 #summer -> heatpump not in use
power_hp = [0] * Time_interval #summer -> heatpump not in use
kwh_per_km = 0.2  # kWh per km driven

### Appliances

In [21]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_summer_case_2,model)

duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_summer_case_2,model)

binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_summer_case_2,model, duration_wm, wm_start)

charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_summer_case_2,model)

pv, load, unmet, curtail, pv_maxed_binary, total_demand,penalty_cost, level_bin, levels, penalty_per_level, demand_level= PV_no_feed_in_and_penalty(Time_interval,
                        merged_data_summer_case_2,model ,power_dishwasher,binary_dishwasher,power_wm,binary_wm,power_dryer,
                        binary_dryer,charging_ev,inflexible_demand,max_power_ev,max_power_hp,power_hp)

### Objective Function

In [22]:
# Electricity cost
electricity_cost = gp.quicksum(price[t] * unmet[t] for t in range(Time_interval))

# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22872 rows, 12094 columns and 65261 nonzeros


Model fingerprint: 0x33bfa741
Variable types: 2688 continuous, 9406 integer (8734 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+04]
  Objective range  [2e-03, 2e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [1e-04, 1e+04]
Presolve removed 17042 rows and 6683 columns
Presolve time: 0.40s
Presolved: 5830 rows, 5411 columns, 28665 nonzeros
Variable types: 896 continuous, 4515 integer (4515 binary)

Root relaxation: objective 2.200795e+01, 3459 iterations, 0.12 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   22.00795    0  382          -   22.00795      -     -    0s
H    0     0                      46.4050219   22.00795  52.6%     -    0s
H    0     0                      46.0629319   22.00795  52.2%     -    0s
H    0     0                      42.5287571   22.00795  48.3%     -    0s
H    0     0                    

### Save Data 

In [23]:
# Prepare results DataFrame
results_df = merged_data_summer_case_2.copy()
results_df['Total_Demand'] = [total_demand[t].getValue() for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
results_df['Unmet'] = [unmet[t].X for t in range(len(results_df))]

# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [demand_level[t].X for t in range(len(results_df))]

# Add penalty cost per hour based on demand level
results_df['Penalty'] = [
    sum(
        penalty_per_level[i] * level_bin[t][i].X
        for i in range(len(penalty_per_level))
    )
    for t in range(len(results_df))
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = [
    price[t] * unmet[t].X for t in range(len(results_df))
]

# Add total cost per hour (electricity + penalty)
results_df['Total_Price'] = [
    results_df['Electricity_Cost'][t] + results_df['Penalty'][t]
    for t in range(len(results_df))
]

# Save to CSV
results_csv_path = os.path.join(project_root + "/results/", "results_case_2_summer.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project/results/results_case_2_summer.csv


# Case 2 Winter 
## PV Generation without feed-in
### Loading Data

In [24]:
merged_data_winter_case_2 = create_df('winter')

c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
c:\Users\Sevi\OneDrive - ETH Zurich\Master Energy Science and Technology\S2\Optimization in Energy Systems\Project\optimization_project\src\create_Dataframe.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Case Global Variables

In [25]:
price = merged_data_winter_case_2["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_winter_case_2['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_2_winter")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 8

### Appliances


In [27]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_winter_case_2,model)

duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_winter_case_2,model)

binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_winter_case_2,model, duration_wm, wm_start)

charging_ev, soc_ev, binary_ev = EV_no_feed_in(Time_interval,merged_data_winter_case_2,model)

power_hp = heat_pump(Time_interval,merged_data_winter_case_2,model,max_power_hp)

pv, load, unmet, curtail, pv_maxed_binary, total_demand,penalty_cost, level_bin, levels, penalty_per_level, demand_level= PV_no_feed_in_and_penalty(Time_interval,
                            merged_data_winter_case_2,model,power_dishwasher,binary_dishwasher
                            ,power_wm,binary_wm,power_dryer,binary_dryer,charging_ev,inflexible_demand,max_power_ev,max_power_hp,power_hp)

### Objective Function

In [ ]:
# Electricity cost
electricity_cost = gp.quicksum(price[t] * unmet[t] for t in range(Time_interval))

# Full objective: cost + penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 38352 rows, 22172 columns and 111030 nonzeros


Model fingerprint: 0xce73bde3
Variable types: 8064 continuous, 14108 integer (13436 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+04]
  Objective range  [2e-04, 2e-01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [5e-05, 1e+04]
Presolve removed 31418 rows and 14162 columns
Presolve time: 0.44s
Presolved: 6934 rows, 8010 columns, 32529 nonzeros
Variable types: 2314 continuous, 5696 integer (5696 binary)
Found heuristic solution: objective 80.1118407

Root relaxation: objective 4.470916e+01, 5027 iterations, 0.13 seconds (0.08 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   44.70916    0  381   80.11184   44.70916  44.2%     -    0s
H    0     0                      60.9957728   44.70916  26.7%     -    0s
H    0     0                      60.8815178   44.70916  26.6%     -    0s
     0     0   45.75923    0  240   60.88152   45.75923 

### Save Results

In [ ]:
# Prepare results DataFrame
results_df = merged_data_winter_case_2.copy()
results_df['Total_Demand'] = [total_demand[t].getValue() for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
results_df['Unmet'] = [unmet[t].X for t in range(len(results_df))]

# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [demand_level[t].X for t in range(len(results_df))]

# Add penalty cost per hour based on demand level
results_df['Penalty'] = [
    sum(
        penalty_per_level[i] * level_bin[t][i].X
        for i in range(len(penalty_per_level))
    )
    for t in range(len(results_df))
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = [
    price[t] * unmet[t].X for t in range(len(results_df))
]

# Add total cost per hour (electricity + penalty)
results_df['Total_Price'] = [
    results_df['Electricity_Cost'][t] + results_df['Penalty'][t]
    for t in range(len(results_df))
]

# Save to CSV
results_csv_path = os.path.join(project_root + "/results/", "results_case_2_winter.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Results saved to /Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/results_case_2_winter.csv


# Case 3 Summer
## V2H/V2G and PV with feed-in
### Loading Data

In [ ]:
merged_data_summer_case_3 = create_df('summer')

/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Case Global Variables

In [ ]:
price = merged_data_summer_case_3["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_summer_case_3['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_3_summer")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 0 #summer -> heatpump not in use
power_hp = [0] * Time_interval #summer -> heatpump not in use
kwh_per_km = 0.2  # kWh per km driven

In [ ]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_summer_case_3,model)

duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_summer_case_3,model)

binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_summer_case_3,model, duration_wm, wm_start)

ev_feed_in_binary,ev_feed_in_power,ev_v2h_feed_in_binary,ev_v2h_power,pv_maxed_binary,unmet,pv_feed_in,total_demand,penalty_cost, level_bin, levels, penalty_per_level, demand_level, unmet = EV_PV_penalty_feed_in_case_3(Time_interval,merged_data_summer_case_3,model,power_dishwasher,binary_dishwasher,power_wm,
                    binary_wm,power_dryer,binary_dryer,kwh_per_km,inflexible_demand,max_power_hp)

NameError: name 'EV_PV_penalty_feed_in_case_3' is not defined

### Objective Function

In [9]:
# electricity cost minus revenue
electricity_cost = gp.quicksum(
        price[t] * unmet[t]
        - ev_feed_in_power[t] * merged_data_summer_case_3['feed_in_tariff'][t]
        - pv_feed_in[t] * merged_data_summer_case_3['feed_in_tariff'][t]
        for t in range(Time_interval))

# Objective: minimize total electricity cost including penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

model.update()
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 55152 rows, 29564 columns and 152118 nonzeros
Model fingerprint: 0x498db361
Variable types: 8064 continuous, 21500 integer (20156 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+04]
  Objective range  [2e-03, 2e-01]
  Bounds range     [1e+00, 7e+01]
  RHS range        [1e-04, 1e+04]


Presolve removed 33340 rows and 11640 columns (presolve time = 12s)...
Presolve removed 33366 rows and 11666 columns
Presolve time: 11.87s
Presolved: 21786 rows, 17898 columns, 83474 nonzeros
Variable types: 6252 continuous, 11646 integer (11646 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.5130651e+02   3.188197e+05   0.000000e+00     12s
    7004   -8.4854570e+00   0.000000e+00   0.000000e+00     12s

Root relaxation: objective -8.485457e+00, 7004 iterations, 0.20 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -8.48546    0  532          -   -8.48546      -     -   12s
H    0     0                      13.8355212   -8.48546   161%     -   12s
H    0     0                      10.7727265   -8.48546   179%     -   13s
H    0     0                       7.8538597   -8.48546   208%  

### Save Data

In [ ]:
# Prepare results DataFrame
results_df = merged_data_summer_case_3.copy()
results_df['Total_Demand'] = [total_demand[t].getValue() for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
results_df['Unmet'] = [unmet[t].X for t in range(len(results_df))]

# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [demand_level[t].X for t in range(len(results_df))]

# Add penalty cost per hour based on demand level
results_df['Penalty'] = [
    sum(
        penalty_per_level[i] * level_bin[t][i].X
        for i in range(len(penalty_per_level))
    )
    for t in range(len(results_df))
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = [
    price[t] * unmet[t].X for t in range(len(results_df))
]

# Add total cost per hour (electricity + penalty)
results_df['Total_Price'] = [
    results_df['Electricity_Cost'][t] + results_df['Penalty'][t]
    for t in range(len(results_df))
]

# Add revenue from PV and EV feed-in
results_df['Revenue'] = [
    (ev_feed_in_power[t].X + pv_feed_in[t].X) * merged_data_summer['feed_in_tariff'][t] 
    for t in range(len(results_df))
]

# Add net price (total price minus revenue)
results_df['Net_Price'] = [
    results_df['Total_Price'][t] - results_df['Revenue'][t]
    for t in range(len(results_df))
]

# Save to CSV
results_csv_path = os.path.join(project_root + "/results/", "results_case_3_summer.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

NameError: name 'total_demand' is not defined

# Case 3 Winter
## V2H/V2G and PV with feed-in
### Loading Data

In [ ]:
merged_data_winter_case_3 = create_df('winter')

/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_summer["timestamp"] = pd.to_datetime(
/Users/simonbernet/Desktop/Optimization in Energy Systems/Optimization Project/optimization_project/src/create_Dataframe.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_winter["timestamp"] = pd.to_datetime(


### Case Global Variables

In [4]:
price = merged_data_winter_case_3["Spotmarket_(EUR/kWh)"].values
inflexible_demand = merged_data_winter_case_3['Inflexible_Demand_(kWh)'].values
Time_interval = len(price)  # Total time interval in hours

# Initiate gurobi model
model = gp.Model("automated_demand_response_case_3_winter")
model.update()

power_dishwasher = 1.5
power_wm = 3
power_dryer = 3
max_power_ev = 10
max_power_hp = 8 
kwh_per_km = 0.2  # kWh per km driven

Set parameter Username
Set parameter LicenseID to value 2653942
Academic license - for non-commercial use only - expires 2026-04-17


### Appliances

In [ ]:
from cases import *

binary_dishwasher, dishwasher_start, start_times = dishwasher(Time_interval,merged_data_winter_case_3,model)

duration_wm, wm_start, binary_wm, start_times_wm = washing_machine(Time_interval,merged_data_winter_case_3,model)

binary_dryer, start_times_dryer, dryer_start = dryer(Time_interval,merged_data_winter_case_3,model, duration_wm, wm_start)

power_hp = heat_pump(Time_interval,merged_data_winter_case_3,model,max_power_hp)

ev_feed_in_binary,ev_feed_in_power,ev_v2h_feed_in_binary,ev_v2h_power,pv_maxed_binary,unmet,pv_feed_in,total_demand,penalty_cost, level_bin, levels, penalty_per_level, demand_level, unmet = EV_PV_penalty_feed_in_case_3(Time_interval,merged_data_winter_case_3,model,power_dishwasher,binary_dishwasher,power_wm,
                    binary_wm,power_dryer,binary_dryer,kwh_per_km,inflexible_demand,power_hp,max_power_hp)

### Objective Function

In [6]:
# electricity cost minus revenue
electricity_cost = gp.quicksum(
        price[t] * unmet[t]
        - ev_feed_in_power[t] * merged_data_winter_case_3['feed_in_tariff'][t]
        - pv_feed_in[t] * merged_data_winter_case_3['feed_in_tariff'][t]
        for t in range(Time_interval))

# Objective: minimize total electricity cost including penalty
model.setObjective(electricity_cost + penalty_cost, GRB.MINIMIZE)

model.update()
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 28920 rows, 16798 columns and 80089 nonzeros
Model fingerprint: 0xa15d860c
Variable types: 6048 continuous, 10750 integer (10078 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+04]
  Objective range  [2e-04, 3e-01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [5e-05, 1e+04]
Presolve removed 16825 rows and 5657 columns
Presolve time: 0.53s
Presolved: 12095 rows, 11141 columns, 45914 nonzeros
Variable types: 4584 continuous, 6557 integer (6557 binary)

Root relaxation: objective 1.674456e+01, 6299 iterations, 0.17 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   16.74456    0  681          -

### Save Data

In [ ]:
# Prepare results DataFrame
results_df = merged_data_winter_case_3.copy()
results_df['Total_Demand'] = [total_demand[t].getValue() for t in range(len(results_df))]
results_df['Dishwasher_Start'] = [dishwasher_start[t].X if t in start_times else 0 for t in range(len(results_df))]
results_df['Dishwasher_On'] = [binary_dishwasher[t].X for t in range(len(results_df))]
results_df['Washing_Machine_Start'] = [wm_start[t].X if t in start_times_wm else 0 for t in range(len(results_df))]
results_df['Washing_Machine_On'] = [binary_wm[t].X for t in range(len(results_df))]
results_df['Dryer_Start'] = [dryer_start[t].X if t in start_times_dryer else 0 for t in range(len(results_df))]
results_df['Dryer_On'] = [binary_dryer[t].X for t in range(len(results_df))]
results_df['EV_SOC'] = [soc_ev[t].X for t in range(len(results_df))]
results_df['EV_Charging'] = [charging_ev[t].X for t in range(len(results_df))]
results_df['EV_On'] = [binary_ev[t].X for t in range(len(results_df))]
results_df['Heatpump_Power'] = [power_hp[t].X for t in range(len(results_df))]
results_df['Unmet'] = [unmet[t].X for t in range(len(results_df))]

# Add demand level index for each hour directly from the Gurobi binary variables
results_df['Demand_Level'] = [demand_level[t].X for t in range(len(results_df))]

# Add penalty cost per hour based on demand level
results_df['Penalty'] = [
    sum(
        penalty_per_level[i] * level_bin[t][i].X
        for i in range(len(penalty_per_level))
    )
    for t in range(len(results_df))
]

# Add electricity cost per hour
results_df['Electricity_Cost'] = [
    price[t] * unmet[t].X for t in range(len(results_df))
]

# Add total cost per hour (electricity + penalty)
results_df['Total_Price'] = [
    results_df['Electricity_Cost'][t] + results_df['Penalty'][t]
    for t in range(len(results_df))
]

# Add revenue from PV and EV feed-in
results_df['Revenue'] = [
    (ev_feed_in_power[t].X + pv_feed_in[t].X) * merged_data_summer['feed_in_tariff'][t] 
    for t in range(len(results_df))
]

# Add net price (total price minus revenue)
results_df['Net_Price'] = [
    results_df['Total_Price'][t] - results_df['Revenue'][t]
    for t in range(len(results_df))
]

# Save to CSV
results_csv_path = os.path.join(project_root + "/results/", "results_case_3_winter.csv")
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")